In [19]:
import numpy as np

import matplotlib.pyplot as plt

from IPython.display import Latex

%matplotlib inline

from NFW import NFW

In [20]:
전data = np.loadtxt("D:/Research/ultra-faint dwarf galaxies/dwarf_dm_5e8_0290.txt")

# 전 is the Hangul character for the surname Jeon of the data gatherer!
# it's just my linguistics side being quirky and I'm glad Jupyter Notebook supports Hangul characters like this!

In [21]:
dx = 전data[:,4]
# collect the 4th element of each row in the array into a massive array

dy = 전data[:,5]
# collect the 5th element of each row in the array into a massive array

dz = 전data[:,6]
# collect the 6th element of each row in the array into a massive array